In [1]:
import mxnet as mx
%matplotlib inline
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import tarfile
import datetime
import warnings
import ROOT as rt
import math
warnings.filterwarnings("ignore", category=DeprecationWarning)

Welcome to JupyROOT 6.10/00


In [2]:
class dataiter(mx.io.DataIter):
    def __init__(self,data_names,data_path,data_shapes,data_gen,label_names,label_path,label_shapes,label_gen,batch_size=100):
        self._provide_data = zip(data_names, data_shapes)
        self._provide_label = zip(label_names, label_shapes)
        self.batch_size = batch_size
        self.data_line=0
        self.data_gen = data_gen
        self.label_gen = label_gen
        self.endfile=0
        self.data_rec=mx.recordio.MXRecordIO(data_path,'r')
        self.label_rec=mx.recordio.MXRecordIO(label_path,'r')
    def __iter__(self):
        return self

    def reset(self):
        self.data_rec.reset()
        self.label_rec.reset()
        self.data_line=0
        self.endfile = 0

    def __next__(self):
        return self.next()

    @property
    def provide_data(self):
        return self._provide_data

    @property
    def provide_label(self):
        return self._provide_label

    def close(self):
        self.record.close()
    def printr(self):
        print self.record.read()
    def next(self):
        if self.endfile==0:
            c=[]
            d=[]
            for i in range(100):
                a=self.data_rec.read()
                b=self.label_rec.read()
                if not a:
                    self.endfile=1
                    break
                self.data_line+=1
                c.append(eval(a))
                d.append(eval(b))
            data=[mx.nd.array(c)]
            label=[mx.nd.array(d)]
            #data = [mx.nd.array(g(d[1])) for d,g in zip(self._provide_data, self.data_gen)]
            #label = [mx.nd.array(g(d[1])) for d,g in zip(self._provide_label, self.label_gen)]
            return mx.io.DataBatch(data, label)
        else:
            raise StopIteration

In [3]:
num_classes = 10
n=32
data_iter = dataiter(['data'], "tmptdb.rec",[(32,32)],
                  [lambda s: np.random.uniform(-1, 1, s)],
                  ['softmax_label'],"tmptlb.rec", [(32,)],
                  [lambda s: np.random.randint(0, num_classes, s)],batch_size=100)

In [31]:
data_iter.provide_data

[('data', (32, 32))]

In [6]:
start=datetime.datetime.now()
jetset=[]
record=mx.recordio.MXRecordIO("tmptdb.rec","r")
while True:
    a=record.read()
    if not a:
        break
    jetset.append(eval(a))
print datetime.datetime.now()-start

0:01:01.700602


In [ ]:
batch.getdata()

In [4]:
start=datetime.datetime.now()
data_iter.reset()
for batch in data_iter:
    print(batch.data)
print datetime.datetime.now()-start

[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArray 100x3x33x33 @cpu(0)>]
[<NDArra

In [5]:
class rootiter(mx.io.DataIter):
    def __init__(self,data_path,data_names,label_names,batch_size=100,endcut=0,arnum=16,maxx=0.4,maxy=0.4):
        self.batchccc=batch_size
        self.batch_size = batch_size
        self._provide_data = zip(data_names, [(self.batch_size, 3, 33, 33)])
        self._provide_label = zip(label_names, [(self.batch_size,)])
        self.ent=0
        self.arnum=arnum
        self.maxx=maxx
        self.maxy=maxy
        self.endfile=0
        self.file=rt.TFile(data_path,'read')
        self.jet=rt.gDirectory.Get('jetAnalyser/jetAnalyser')
        self.Entries=self.jet.GetEntriesFast()
    def __iter__(self):
        return self

    def reset(self):
        selfbatchccc=self.batch_size
        self.jet.GetEntry(0)
        self.ent=0
        self.endfile = 0

    def __next__(self):
        return self.next()

    @property
    def provide_data(self):
        return self._provide_data

    @property
    def provide_label(self):
        return self._provide_label

    def close(self):
        self.file.Close()
    def printr(self):
        print self.ent,self.jet.pt
    def next(self):
        if self.endfile==0:
            maxx=self.maxx
            maxy=self.maxy
            arnum=self.arnum
            bx=2*maxx/(2*arnum+1)
            by=2*maxy/(2*arnum+1)
            jetset=[]
            labels=[]
            for i in range(self.batch_size):
                if self.ent>=self.Entries:
                    self.endfile=1
                    self.ent=0
                    #raise StopIteration
                    #break
                self.jet.GetEntry(self.ent)
                if self.jet.partonId==0:
                    continue
                palet=np.zeros(shape=(3,2*arnum+1,2*arnum+1))
                r=0
                g=0
                b=0
                ##
                for i in range(len(self.jet.dau_pt)):
                    x=int(math.floor((self.jet.dau_deta[i]/bx)+0.5)+arnum)
                    if x<0 or x>2*arnum:
                        continue
                    y=int(math.floor((self.jet.dau_dphi[i]/by)+0.5)+arnum)
                    if y<0 or y>2*arnum:
                        continue
                    pt=self.jet.dau_pt[i]
                    if self.jet.dau_charge[i]==0:
                        palet[1][x][y]+=pt
                        #overlap[1][x][y]+=pt
                        if palet[1][x][y]>g:
                            g=palet[1][x][y]
                    else:
                        palet[0][x][y]+=pt
                        palet[2][x][y]+=1
                        #overlap[0][x][y]+=pt
                        #overlap[2][x][y]+=1
                        if palet[0][x][y]>r:
                            r=palet[0][x][y]
                        if palet[2][x][y]>b:
                            b=palet[2][x][y]
                        #pass
                #record.write(palet)
                for i in range(2*arnum+1):
                    for j in range(2*arnum+1):
                        if r!=0:
                            palet[0][i][j]=255*palet[0][i][j]/r
                        if g!=0:
                            palet[1][i][j]=255*palet[1][i][j]/g
                        if b!=0:
                            palet[2][i][j]=255*palet[2][i][j]/b
                ##
                jetset.append(palet)
                
                if self.jet.partonId==21:
                    labels.append(1)
                else:
                    labels.append(0)
                self.ent+=1
                if(self.endfile!=0):
                    if (len(labels)>=self.batch_size):
                        self.ent=0
                        break
            self.batccc=len(labels)
            data=[mx.nd.array(jetset)]
            label=[mx.nd.array(labels)]
            #data = [mx.nd.array(g(d[1])) for d,g in zip(self._provide_data, self.data_gen)]
            #label = [mx.nd.array(g(d[1])) for d,g in zip(self._provide_label, self.label_gen)]
            return mx.io.DataBatch(data, label)
        else:
            raise StopIteration

In [16]:
int(100123*1.0)

100123

In [6]:
root_iter = rootiter( "jet2.root",['data'],['softmax_label'],batch_size=1000)

In [62]:
root_iter.reset()
root_iter.next()

In [63]:
root_iter.provide_data

[('data', (100, 3L, 33L, 33L))]

In [ ]:
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
0:01:14.475702

In [ ]:
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
0:01:12.295478

In [7]:
start=datetime.datetime.now()
root_iter.reset()
for batch in root_iter:
    print(batch.data)
print datetime.datetime.now()-start

[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
[<NDArray 1000x3x33x33 @cpu(0)>]
0:01:12.295478


In [24]:
root_iter.next()

In [25]:
root_iter.printr()

100 123.317497253


In [21]:
root_iter.reset()

In [4]:
import mxnet as mx
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, auc, precision_recall_curve, roc_curve, average_precision_score

#data_train=data1[:int(len(data1)*0.7)]
#data_test=data1[int(len(data1)*0.7):]
#label_train=labels[:int(len(labels)*0.7)]
#label_test=labels[int(len(labels)*0.7):]

data = mx.sym.var('data')
# first conv layer
conv1 = mx.sym.Convolution(data=data, kernel=(5,5), num_filter=20)
tanh1 = mx.sym.Activation(data=conv1, act_type="relu")
pool1 = mx.sym.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
# second conv layer
conv2 = mx.sym.Convolution(data=pool1, kernel=(5,5), num_filter=50)
tanh2 = mx.sym.Activation(data=conv2, act_type="relu")
pool2 = mx.sym.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
# first fullc layer
flatten = mx.sym.flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
tanh3 = mx.sym.Activation(data=fc1, act_type="relu")
# second fullc
fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=2)
# softmax loss
lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')
#mx.viz.plot_network(lenet)

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout
# create a trainable module on GPU 0
lenet_model = mx.mod.Module(symbol=lenet, context=mx.cpu())
# train with the same 
"""
batch_end_callback = mx.callback.Speedometer(batch_size, 1000),
optimizer_params={'learning_rate':0.1},
"""
#optimizer_params={'learning_rate':0.5,'beta1':0.1,'beta2':0.111},
lenet_model.fit(root_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate':0.1},
                eval_metric='acc',
                batch_end_callback = mx.callback.Speedometer(100, 100),
                num_epoch=10)

INFO:root:Epoch[0] Train-accuracy=0.629128
INFO:root:Epoch[0] Time cost=104.086
INFO:root:Epoch[1] Train-accuracy=0.665772
INFO:root:Epoch[1] Time cost=84.357
INFO:root:Epoch[2] Train-accuracy=0.677383
INFO:root:Epoch[2] Time cost=82.980


In [68]:
import mxnet as mx
data=mx.sym.var('data')
conv1 = mx.sym.Convolution(data=data, kernel=(3,3),pad=(1,1), num_filter=64)
relu1 = mx.sym.Activation(data=conv1, act_type="relu")
pool1 = mx.sym.Pooling(data=relu1, pool_type="max", kernel=(2,2), stride=(2,2))
conv2 = mx.sym.Convolution(data=pool1, kernel=(3,3),pad=(1,1), num_filter=128)
relu2 = mx.sym.Activation(data=conv2, act_type="relu")
pool2 = mx.sym.Pooling(data=relu2, pool_type="max", kernel=(2,2), stride=(2,2))
conv3 = mx.sym.Convolution(data=pool2, kernel=(3,3),pad=(1,1), num_filter=256)
relu3 = mx.sym.Activation(data=conv3, act_type="relu")
conv4 = mx.sym.Convolution(data=relu3, kernel=(3,3),pad=(1,1), num_filter=256)
relu4 = mx.sym.Activation(data=conv4, act_type="relu")
pool3 = mx.sym.Pooling(data=relu4, pool_type="max", kernel=(2,2), stride=(2,2))
flatten = mx.sym.flatten(data=pool3)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=4096)
relu9 = mx.sym.Activation(data=fc1, act_type="relu")
drop1=mx.sym.Dropout(data=relu9)
fc2 = mx.symbol.FullyConnected(data=drop1, num_hidden=4096)
relu10 = mx.sym.Activation(data=fc2, act_type="relu")
drop2=mx.sym.Dropout(data=relu10)
fc3 = mx.symbol.FullyConnected(data=drop2, num_hidden=2)
vggnet=mx.sym.SoftmaxOutput(data=fc3, name='softmax')
#mx.viz.plot_network(vggnet,shape={"data":(1,3,33,33)})
print "vgg"

import logging
logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout
# create a trainable module on GPU 0
lenet_model = mx.mod.Module(symbol=vggnet, context=mx.gpu())
# train with the same 
"""
batch_end_callback = mx.callback.Speedometer(batch_size, 1000),
optimizer_params={'learning_rate':0.1},
"""
#optimizer_params={'learning_rate':0.5,'beta1':0.1,'beta2':0.111},
lenet_model.fit(train_iter,
                eval_data=val_iter,
                optimizer='sgd',
                optimizer_params={'learning_rate':0.1},
                eval_metric='acc',
                batch_end_callback = mx.callback.Speedometer(batch_size, 1000),
                num_epoch=10)

long